In [1]:
import os
import pdfplumber
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.tools import Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document

In [5]:
from labagent import lab_tool

ModuleNotFoundError: No module named 'labagent'

In [ ]:
apikey = "YOURAPIKEY" #get it from MISTRALAI
llm = ChatOpenAI(
    openai_api_key=apikey,
    openai_api_base="https://api.mistral.ai/v1",
    model="ministral-8b-latest"
    
)

C:\Users\Varun Gupta\AppData\Local\Temp\ipykernel_5264\2828721541.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [5]:
def load_pdfs(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            with pdfplumber.open(file_path) as pdf:
                text = "".join(page.extract_text() or "" for page in pdf.pages)
                if text.strip():
                    documents.append(Document(page_content=text))
    return documents


    return documents
docs = load_pdfs("documents")

In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
documents = text_splitter.split_documents(docs)


In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


NameError: name 'HuggingFaceEmbeddings' is not defined

In [46]:
faiss_index = FAISS.from_documents(documents, embedding_model)
faiss_index.save_local("faiss_index")

In [18]:
retriever = faiss_index.as_retriever(search_kwargs={"k": 3})


In [15]:
faiss_index = FAISS.load_local(
    "faiss_index", 
    embeddings=embedding_model, 
    allow_dangerous_deserialization=True
)


In [19]:
query = "prevention tips for diabetes?"
retrieved_docs = retriever.get_relevant_documents(query)

# Display retrieved documents
for idx, doc in enumerate(retrieved_docs):
    print(f"Document {idx + 1}:\n{doc.page_content}\n{'-'*50}")


Document 1:
• Choose whole grains. Whole-grain breads,
• LDL Your particular LDL target depends on pasta, and cereal help prevent a blood sugar
--------------------------------------------------
Document 2:
glycemia awareness, foot and eye care
• Mild cases may be controlled initially with diet, exercise, and
weight loss
• Metformin or alternative oral agents if diet is ineffective; insulin
--------------------------------------------------
Document 3:
Page 10 www.health.harvard.edu • 101 TIPS FOR TIP-TOP HEALTH
ThinksTock
ThinksTock• Add a small amount of healthy fat. Sprinkle on lunch, now’s your chance; why not load up on fruits
--------------------------------------------------


C:\Users\Varun Gupta\AppData\Local\Temp\ipykernel_11972\2586591605.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [20]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Encode query and documents
query_embedding = np.array(embedding_model.embed_query(query)).reshape(1, -1)
doc_embeddings = np.array([embedding_model.embed_query(doc.page_content) for doc in retrieved_docs])

# Compute similarity scores
similarity_scores = cosine_similarity(query_embedding, doc_embeddings)

# Display scores
for idx, score in enumerate(similarity_scores[0]):
    print(f"Document {idx + 1} SS Score: {score:.4f}")


Document 1 SS Score: 0.7176
Document 2 SS Score: 0.6916
Document 3 SS Score: 0.6793


In [34]:
prompt_template = PromptTemplate(
    input_variables=["name", "age", "medical_history", "history", "input"],
    template="""You are a caring and intelligent healthcare assistant designed to provide accurate medical advice tailored to the user's personal details. 
You prioritize warmth, empathy, and factual information.

### Greeting:
Greet the user warmly by name, acknowledging their ongoing health journey. If appropriate, mention their conditions to show attentiveness.
Also you are directly talking to the user.So use You/Your/We instead he/him/they/{name}.
### User Profile:
- **Name:** {name}
- **Age:** {age}
- **Medical History:** {medical_history}

### Conversation History:
{history}

### User Query:
{input}

### Instructions:
- Start with a friendly greeting like "Hello {name}, I hope you're feeling well today!" or "Hi {name}, I'm here to help you with anything you need."
- When answering medical queries, thoughtfully consider the user's medical history.
- Proactively suggest additional health tips that align with their conditions.
- Provide practical, actionable advice based on their age, lifestyle, and medical conditions.
- Maintain a positive and encouraging tone to support their well-being.
- If unsure, recommend consulting a healthcare professional for detailed guidance.

Your response:"""
)


In [35]:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# RAG Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [41]:
search_tool = DuckDuckGoSearchRun()

# Tools for the Agent
tools = [
    Tool(
        name="HealthcareRAG",
        func=qa_chain.run,
        description="Useful for answering healthcare-related questions."
    ),
    Tool(
        name="WebSearch",
        func=search_tool.run,
        description="Useful for fetching up-to-date healthcare information from the web."
    ),lab_tool
]


In [42]:
chatbot = initialize_agent(
    tools=tools,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    llm=llm,
    verbose=True,
    memory=memory,
    prompt=prompt_template
)


In [38]:
name = "John"
age = 30
med_his = "Type 2 diabetes, allergy, liver patient"
message = "which exercise to pratice as a patient"

combined_input = f"""
User Profile:
- Name: {name}
- Age: {age}
- Medical History: {med_his}

User Query: {message}
"""

chatbot.invoke({
    "input": combined_input
})




> Entering new AgentExecutor chain...
To determine the best exercise for John, who has Type 2 diabetes, an allergy, and is a liver patient, I need to consider his medical conditions and overall health. I'll use the HealthcareRAG tool to get specific recommendations for exercises suitable for his conditions.

Action: HealthcareRAG
Action Input: "What are the best exercises for a 30-year-old with Type 2 diabetes, an allergy, and liver disease?"
Observation: Based on the provided context, here are some exercise recommendations for a 30-year-old with Type 2 diabetes, an allergy, and liver disease:

1. **Low-Impact Aerobic Exercises**: These are generally safe and effective for people with liver disease. Examples include:
   - Brisk walking
   - Cycling (stationary or outdoors)
   - Swimming
   - Water aerobics

2. **Strength Training**: Exercises that strengthen the muscles around the joints can help with weight management and overall health. Focus on exercises that do not involve heavy 

{'input': '\nUser Profile:\n- Name: John\n- Age: 30\n- Medical History: Type 2 diabetes, allergy, liver patient\n\nUser Query: which exercise to pratice as a patient\n',
 'history': [HumanMessage(content='\nUser Profile:\n- Name: John\n- Age: 30\n- Medical History: Type 2 diabetes, allergy, liver patient\n\nUser Query: which exercise to pratice as a patient\n', additional_kwargs={}, response_metadata={}),
  AIMessage(content='John should focus on low-impact aerobic exercises, strength training, and flexibility and balance exercises. He should avoid high-impact exercises and always consult with a healthcare provider before starting any new exercise program.', additional_kwargs={}, response_metadata={})],
 'output': 'John should focus on low-impact aerobic exercises, strength training, and flexibility and balance exercises. He should avoid high-impact exercises and always consult with a healthcare provider before starting any new exercise program.'}